In [ ]:
from datetime import datetime
current_datetime = datetime.now()
datetime_string = current_datetime.strftime("%Y_%m_%d_%H_%M_%S")
hardware_filename = "bc_" + datetime_string
hardware_filename

In [ ]:
def hardwareImage(pix):
    modifiedPixel = 336 - pix*20
    return modifiedPixel

def softwareImage(modifiedPixel):
    pix = (336 - modifiedPixel) / 20
    return pix

num_data_points = len(mnist_dataset.get_test_data())
test_loader_h = DataLoader(mnist_dataset.get_test_data(), batch_size=num_data_points, shuffle=True, drop_last=True)

In [ ]:
fr_np = []

for data, targets in test_loader_h:
    data = data.to(device)
    data_preSNN = data
    data = spikegen.latency(data, num_steps=modelConstants.num_steps, threshold=0.01, clip=True, first_spike_time=0, linear=True, normalize=True).cumsum(0)

    if(modelConstants.pretrained_weights_used):
        with torch.no_grad():  # Disable gradient tracking
            net.fc1.linear.weight.data.copy_(trained_weights)

    spk_rec, mem = net(data)
    #fr = (spk_rec.sum(0) / spk_rec.shape[0])
    fr = spk_rec.sum(0)
    summed_data = torch.sum(data, axis=0)
print(data.shape)
print(summed_data.shape, num_data_points)

hardwareInput = hardwareImage(summed_data)

inputData = np.zeros((num_data_points, modelConstants.num_inputs))

for k in range(num_data_points):
    temp_image = hardwareInput[k]
    flat_image = temp_image.view(-1)

    for pixel in range (flat_image.numel()):
        inputData[k][pixel] = flat_image[pixel]

test_data = inputData.T

filename_inData = f'{results_path}/inputData{num_data_points}39ideal_v{counter_global}.csv'
with open(filename_inData,'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(test_data)

for elem in fr.view(-1):
    fr_np.append(elem.detach().cpu().item())

firing_rate = np.array(fr_np).reshape(-1, 1)

filename = f'{results_path}/firing_rate{num_data_points}39ideal_v{counter_global}.csv'
with open(filename,'w') as h:
    csv.writer(h).writerows(firing_rate)
counter_global += 1

In [ ]:
# generating text file for a batch of test inputs
filename = f'{results_path}/sim_{num_samples}_v{counter_global}.txt'
fout = open(filename, "wt")
fout.write("ii = 0 \n")

for k in range(num_samples):
    fout.write('''simulator( 'spectre )
design( "DigNeuron_New" "EntireNeuronArchV2SingleCell_Rezvan" "schematic")
resultsDir( "/data/shared/brainer/home/zmoham12/simulation/DigNeuron_New/EntireNeuronArchV2SingleCell_Rezvan/maestro/results/maestro/.tmpADEDir_zahra.mohammadrezaee/DigNeuron_New_EntireNeuronArchV2SingleCell_1/simulation/EntireNeuronArchV2SingleCell_Rezvan/spectre/schematic" )
modelFile( 
    '("$SPECTRE_MODEL_PATH/design_wrapper.lib.scs" "tt_pre")
)
analysis('tran ?stop "340n"  )
desVar(	  "VG" 110m	)
desVar(	  "VD" 0.8	)
desVar(	  "f" 20n	)
''')
    temp_image = hardwareInput[k]
    flat_image = temp_image.view(-1)

    for pixel in range (flat_image.numel()):
        A = 'desVar(       "Vd{}" {}n )'
        fout.write(A.format(pixel,flat_image[pixel]) + "\n")

    fout.write(f'''
desVar(	  "wx" 0m	)
envOption(
	'analysisOrder  list("pz" "dcmatch" "stb" "tran" "envlp" "ac" "dc" "lf" "noise" "xf" "sp" "pss" "pac" "pstb" "pnoise" "pxf" "psp" "qpss" "qpac" "qpnoise" "qpxf" "qpsp" "hb" "hbac" "hbstb" "hbnoise" "hbxf" "sens" "acmatch")
	'switchViewList  list("spectre" "spectreText" "cmos_sch" "cmos.sch" "schematic" "veriloga")
)
option(	'pzSeverity  "None"
	'noiseSeverity  "None"
	'spSeverity  "None"
	'acSeverity  "None"
	'dcOpSeverity  "None"
	'dcSeverity  "None"
	'tranSeverity  "None"
)
temp( 27 )
out = outfile("results{hardware_filename}.txt" "a")
run()
selectResult( 'tran )
outputs()
ocnPrint(?output out sample( VT("/Fire") 0 340n "linear" 1n) )
close(out)
for( i 1 30  printf( "%d \\n" ii ))
ii = ii+1
''')
fout.close()

In [ ]:
test_data = np.genfromtxt('C:/Users/rezva/Documents/hardwareAwareLearning/results/inputData2019_v2.csv', delimiter=',', dtype=np.float32)
inputData = test_data.T
total_elements = inputData.size
num_data_points = total_elements // (modelConstants.image_size * modelConstants.image_size)

if total_elements % (modelConstants.image_size * modelConstants.image_size) != 0:
    raise ValueError("Input data cannot be reshaped into the desired dimensions. Check the CSV file format.")
hardwareInput_reconstructed = torch.tensor(inputData).view(num_data_points, 1, modelConstants.image_size * modelConstants.image_size)
print("Reconstructed tensor shape:", hardwareInput_reconstructed.shape)
